In [1]:
! pip install autogen proton_driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.4/979.4 kB 13.6 MB/s eta 0:00:0000:010:01


### Define Timeplus Tools

In [2]:
import os

from proton_driver import client

timeplus_host = os.getenv("TIMEPLUS_HOST") or "localhost"
timeplus_user = os.getenv("TIMEPLUS_USER") or "proton"
timeplus_password = os.getenv("TIMEPLUS_PASSWORD") or "timeplus@t+"

class Tools:
    def __init__(self) -> None:
        self.client = client.Client(host=timeplus_host, port=8463, user=timeplus_user,password=timeplus_password)

    def list_table(self, *args):
        result = []
        rows = self.client.execute_iter("SHOW STREAMS")
        for row in rows:
            result.append(row[0])
        return result
    
    def describe_table(self, *args):
        name = args[0]
        result = []
        rows = self.client.execute_iter(f"DESCRIBE {name.strip()}")
        for row in rows:
            col = {}
            col["name"] =  row[0]
            col["type"] =  row[1]
            result.append(col)
        return result

    def run(self, tool_name, *args):
        result = getattr(self, tool_name)(*args)
        return result

    def list(self):
        return ["list_table", "describe_table"]

In [3]:
tool = Tools()

tool.list_table()

tool.describe_table('kafka_cdc_postgres_customers')

[{'name': 'raw', 'type': 'string'}]

In [6]:
def list_table() -> Annotated[str, "The name of tables in the system"]:
    return tool.list_table()

def describe_table(name: Annotated[str, "The name of the table"]) -> Annotated[str, "schema definition of the table"]:
    return tool.describe_table(name)
        

In [5]:
import os
from typing import Annotated

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function
from autogen.agentchat.contrib.capabilities import teachability
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

config_list = [
    {"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]},
]

In [7]:
# NOTE: this ReAct prompt is adapted from Langchain's ReAct agent: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/react/agent.py#L79
ReAct_prompt = """
You are a asistent help generating SQL based on input questions. 
Please stop when you have the SQL, no need to execute the SQL
To generate SQL, here are rules:
* the grammar follows ClickHouse style
* all datatypes MUST be in lowercase, such uint32
* all keywords MUST be in lowercase, such as nullable
* for normal query, add table() function to the table name, for example select count(*) from table(table_name)
* for real time query, where continously return new result to the user, append a time range, for example
  select count(*) from table_name where _tp_time > now() -1h
  which will return the number of event received in the past 1 hour

You have access to tools provided.
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context


def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

In [8]:
# Setting up code executor.
os.makedirs("coding", exist_ok=True)
# Use docker executor for running code in a container if you have docker installed.
# code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    ##code_execution_config={"executor": code_executor},
)

assistant = AssistantAgent(
    name="Assistant",
    system_message=(
        "Only use the tools you have been provided with."
        "You are a SQL generator. Your task is to generate SQL queries and nothing else. "
        "Do NOT execute SQL or ask for execution. Once the SQL is generated, respond with TERMINATE."
    ),
    llm_config={"config_list": config_list, "cache_seed": None},
)

# Register the timeplus tool.
register_function(
    list_table,
    caller=assistant,
    executor=user_proxy,
    name="list_table",
    description="list available tables in the system",
)

register_function(
    describe_table,
    caller=assistant,
    executor=user_proxy,
    name="describe_table",
    description="return the schema of the table",
)

# Cache LLM responses. To get different responses, change the cache_seed value.
with Cache.disk(cache_seed=43) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question="how many customer are there in the past 1 day in real time",
        cache=cache,
    )

    print(f" the final answer is { user_proxy.last_message()['content'] } ")

User (to Assistant):


You are a asistent help generating SQL based on input questions. 
Please stop when you have the SQL, no need to execute the SQL
To generate SQL, here are rules:
* the grammar follows ClickHouse style
* all datatypes MUST be in lowercase, such uint32
* all keywords MUST be in lowercase, such as nullable
* for normal query, add table() function to the table name, for example select count(*) from table(table_name)
* for real time query, where continously return new result to the user, append a time range, for example
  select count(*) from table_name where _tp_time > now() -1h
  which will return the number of event received in the past 1 hour

You have access to tools provided.
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know th